In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
from konlpy.tag import Mecab
from copy import deepcopy
tagger = Mecab()
flatten = lambda l: [item for sublist in l for item in sublist]
from data_utils import *
from model import SDEN
import pickle

In [2]:
word2index = pickle.load(open('vocab.pkl','rb'))
slot2index = pickle.load(open('slot.pkl','rb'))
intent2index = pickle.load(open('intent.pkl','rb'))

In [11]:
index2intent = {v:k for k,v in intent2index.items()}
index2slot = {v:k for k,v in slot2index.items()}

In [3]:
model = SDEN(len(word2index),100,64,len(slot2index),len(intent2index))
model.load_state_dict(torch.load('sden.pkl'))

In [54]:
model.eval()

SDEN(
  (embed): Embedding(14377, 100)
  (bigru_m): GRU(100, 64, batch_first=True, bidirectional=True)
  (bigru_c): GRU(100, 64, batch_first=True, bidirectional=True)
  (context_encoder): Sequential(
    (0): Linear(in_features=256, out_features=128, bias=True)
    (1): Sigmoid()
  )
  (session_encoder): GRU(128, 128, batch_first=True, bidirectional=True)
  (decoder_1): GRU(100, 128, batch_first=True, bidirectional=True)
  (decoder_2): LSTM(256, 128, batch_first=True, bidirectional=True)
  (intent_linear): Linear(in_features=256, out_features=12, bias=True)
  (slot_linear): Linear(in_features=256, out_features=32, bias=True)
  (dropout): Dropout(p=0.5)
)

In [67]:
history = ["스케줄 추가해줘","날짜를 알려주세요."]
current = "나 근데 꽃배달해줘"

In [68]:
history = [tagger.morphs(h) for h in history]
current = tagger.morphs(current)

In [69]:
history = [prepare_sequence(h,word2index).view(1,-1) for h in history]
current = prepare_sequence(current,word2index).view(1,-1)
history = pad_to_history(history,word2index)

s,i = model(history,current)
slot_p = s.max(1)[1]
intent_p = i.max(1)[1]
print(index2intent[intent_p.data[0]])
print([index2slot[s] for s in slot_p.data.tolist()])

FLOWER_Meta
['O', 'O', 'O', 'O', 'O', 'O']
